# 参考
https://qiita.com/Hyperion13fleet/items/4eaca365f28049fe11c7  
https://qiita.com/daigomiyoshi/items/d6799cc70b2c1d901fb5


In [1]:
import featuretools as ft
import pandas as pd

df_cat = pd.DataFrame({'cat_id': [1, 2],
                   'X': [11, 12]})

df = pd.DataFrame({'cat_id': [1, 1, 2, 2],
                   'B': [11, 12, 13, 14],
                   'C': [21, 22, 23, 24],
                   'D': [31, 32, 33, 34],
                   't': ['2014-01-01 00:00:00', '2014-01-02 00:00:00', '2014-01-03 00:00:00', '2014-01-04 00:00:00']}
                  )
df = df.reset_index()
df = df.astype({'C': float, 'D': float})
df

,index,cat_id,B,C,D,t
0,0,1,11,21.0,31.0,2014-01-01 00:00:00
1,1,1,12,22.0,32.0,2014-01-02 00:00:00
2,2,2,13,23.0,33.0,2014-01-03 00:00:00
3,3,2,14,24.0,34.0,2014-01-04 00:00:00


In [2]:
df.dtypes

index       int64
cat_id      int64
B           int64
C         float64
D         float64
t          object
dtype: object

# 基本

In [3]:
# generate EntitySet
es = ft.EntitySet(id='test1')

es.entity_from_dataframe(entity_id='ent1', dataframe=df, index='index', time_index='t')
# es.entity_from_dataframe(entity_id='cat1', dataframe=df_cat)

# relation = ft.Relationship(es['ent1']['cat_id'], es['cat1']['cat_id'])
# es.add_relationships(relationships=[relation])

es['ent1'].variables



[<Variable: index (dtype = index)>,
 <Variable: cat_id (dtype = numeric)>,
 <Variable: B (dtype = numeric)>,
 <Variable: C (dtype = numeric)>,
 <Variable: D (dtype = numeric)>,
 <Variable: t (dtype: datetime_time_index, format: None)>]

In [4]:
list_agg = ['sum','min'] 
list_trans = ['add_numeric', 'subtract_numeric', 'multiply_numeric', 'divide_numeric', 'year']

dfa, _ = ft.dfs(entityset=es,
                target_entity='ent1',
                agg_primitives=list_agg,
                trans_primitives =list_trans,
                max_depth=3)

# count features
print(dfa.shape)
print(list(dfa.columns))
dfa.head()

(4, 35)
['cat_id', 'B', 'C', 'D', 'B + C', 'B + D', 'B + cat_id', 'C + D', 'C + cat_id', 'cat_id + D', 'B / C', 'B / D', 'B / cat_id', 'C / B', 'C / D', 'C / cat_id', 'D / B', 'D / C', 'D / cat_id', 'cat_id / B', 'cat_id / C', 'cat_id / D', 'B * C', 'B * D', 'B * cat_id', 'C * D', 'C * cat_id', 'cat_id * D', 'B - C', 'B - D', 'B - cat_id', 'C - D', 'C - cat_id', 'cat_id - D', 'YEAR(t)']


c:\users\neko9\desk\dev\.venv\lib\site-packages\featuretools\synthesis\dfs.py:317: UnusedPrimitiveWarning: Some specified primitives were not used during DFS:
  agg_primitives: ['min', 'sum']
This may be caused by a using a value of max_depth that is too small, not setting interesting values, or it may indicate no compatible variable types for the primitive were found in the data.
  warnings.warn(warning_msg, UnusedPrimitiveWarning)


,cat_id,B,C,D,B + C,B + D,B + cat_id,C + D,C + cat_id,cat_id + D,...,C * D,C * cat_id,cat_id * D,B - C,B - D,B - cat_id,C - D,C - cat_id,cat_id - D,YEAR(t)
index,,,,,,,,,,,,,,,,,,,,,
0,1,11,21.0,31.0,32.0,42.0,12,52.0,22.0,32.0,...,651.0,21.0,31.0,-10.0,-20.0,10,-10.0,20.0,-30.0,2014
1,1,12,22.0,32.0,34.0,44.0,13,54.0,23.0,33.0,...,704.0,22.0,32.0,-10.0,-20.0,11,-10.0,21.0,-31.0,2014
2,2,13,23.0,33.0,36.0,46.0,15,56.0,25.0,35.0,...,759.0,46.0,66.0,-10.0,-20.0,11,-10.0,21.0,-31.0,2014
3,2,14,24.0,34.0,38.0,48.0,16,58.0,26.0,36.0,...,816.0,48.0,68.0,-10.0,-20.0,12,-10.0,22.0,-32.0,2014


# さらにfeaturetoolsを実行

In [5]:
dfa = dfa.reset_index()
print(dfa.head())
# generate EntitySet
es2 = ft.EntitySet(id='test2')
es.entity_from_dataframe(entity_id='ent2', dataframe=dfa, index='index')

list_agg = ['sum','min'] 
list_trans = ['add_numeric', 'multiply_numeric']

dfb, _ = ft.dfs(entityset=es,
                target_entity='ent2',
                agg_primitives=list_agg,
                trans_primitives =list_trans,
                max_depth=3)
# count features
print(dfb.shape)
print(list(dfb.columns))
dfb.head()

   index  cat_id   B     C     D  B + C  B + D  B + cat_id  C + D  C + cat_id  \
0      0       1  11  21.0  31.0   32.0   42.0          12   52.0        22.0   
1      1       1  12  22.0  32.0   34.0   44.0          13   54.0        23.0   
2      2       2  13  23.0  33.0   36.0   46.0          15   56.0        25.0   
3      3       2  14  24.0  34.0   38.0   48.0          16   58.0        26.0   

   ...  C * D  C * cat_id  cat_id * D  B - C  B - D  B - cat_id  C - D  \
0  ...  651.0        21.0        31.0  -10.0  -20.0          10  -10.0   
1  ...  704.0        22.0        32.0  -10.0  -20.0          11  -10.0   
2  ...  759.0        46.0        66.0  -10.0  -20.0          11  -10.0   
3  ...  816.0        48.0        68.0  -10.0  -20.0          12  -10.0   

   C - cat_id  cat_id - D  YEAR(t)  
0        20.0       -30.0     2014  
1        21.0       -31.0     2014  
2        21.0       -31.0     2014  
3        22.0       -32.0     2014  

[4 rows x 36 columns]
2020-12-13 14:1

,cat_id,B,C,D,B + C,B + D,B + cat_id,C + D,C + cat_id,cat_id + D,...,cat_id / C * D / B,cat_id / C * D / C,cat_id / C * D / cat_id,cat_id / C * YEAR(t),cat_id / C * cat_id / D,cat_id / D * D,cat_id / D * D / B,cat_id / D * D / C,cat_id / D * D / cat_id,cat_id / D * YEAR(t)
index,,,,,,,,,,,,,,,,,,,,,
0,1,11,21.0,31.0,32.0,42.0,12,52.0,22.0,32.0,...,0.134199,0.070295,1.476190,95.904762,0.001536,1.0,0.090909,0.047619,1.0,64.967742
1,1,12,22.0,32.0,34.0,44.0,13,54.0,23.0,33.0,...,0.121212,0.066116,1.454545,91.545455,0.001420,1.0,0.083333,0.045455,1.0,62.937500
2,2,13,23.0,33.0,36.0,46.0,15,56.0,25.0,35.0,...,0.220736,0.124764,1.434783,175.130435,0.005270,2.0,0.153846,0.086957,1.0,122.060606
3,2,14,24.0,34.0,38.0,48.0,16,58.0,26.0,36.0,...,0.202381,0.118056,1.416667,167.833333,0.004902,2.0,0.142857,0.083333,1.0,118.470588
